# Compatbilização das malhas censitárias de 2010 e 2022 para a Região Metropolitana de São Paulo

In [1]:
from utils import *

In [2]:
BASENAME = '2010-2022-RMSP' # Nome da compatibilização
NAME_C1 = '2010' # Nome da primeira malha censitária
NAME_C2 = '2022' # Nome da segunda malha censitária

## 1. Configuração das malhas

In [3]:
# Importação
malha_2010 = gpd.read_parquet(f'malhas/{NAME_C1}.parquet')
malha_2022 = gpd.read_parquet(f'malhas/{NAME_C2}.parquet')

## 2. Criação do grafo de compatibilização

In [4]:
G_compat = compatibility_graph(malha_2010, malha_2022)

/home/pedrorm/Documentos/projetos/compat_censos/utils.py:301: UserWarning: `keep_geom_type=True` in overlay resulted in 182706 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  self.gridUnion = gpd.overlay(gA, gB, how='union')\


In [5]:
G_compat.compatManutencao()
G_compat.compatDivisao(threshold=0.8)

# Aplica séries de buffers
for b in [-20, -10]:
    G_compat.compatSobreposicao(buffer=b)

# Sobreposição de isolados restantes
G_compat.compatSobreposicao(buffer=0, use_all=True)

G_compat.exportCompatFiles(BASENAME, NAME_C1, NAME_C2)

/home/pedrorm/anaconda3/envs/compat_censos/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/pedrorm/Documentos/projetos/compat_censos/utils.py:377: UserWarning: `keep_geom_type=True` in overlay resulted in 282 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersecao = gpd.overlay(isolados_A, buffered_B, how='union')\
/home/pedrorm/anaconda3/envs/compat_censos/lib/python3.13/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

## 3. Métricas dos resultados

In [6]:
df = pd.DataFrame([i[1] for i in G_compat.nodes(data=True)])

In [7]:
pd.DataFrame([i[1] for i in G_compat.nodes(data=True)])

,malha,nome,group,geom,center,classe,grau
0,A,350390105000001,35039010500,MULTIPOLYGON (((365372.93582581804 7412160.522...,POINT (365067.63153716014 7412047.847474802),Exclusão,2
1,A,350390105000002,35039010500,MULTIPOLYGON (((365164.2863707787 7412334.3723...,POINT (364973.2061534119 7412437.8464869745),Manutenção,1
2,A,350390105000003,35039010500,MULTIPOLYGON (((365309.2869993468 7412485.3730...,POINT (365357.7114953455 7412377.231328052),Manutenção,1
3,A,350390105000004,35039010500,MULTIPOLYGON (((365551.1522835331 7412059.3480...,POINT (365482.28521071794 7411732.847325092),Manutenção,1
4,A,350390105000005,35039010500,MULTIPOLYGON (((364962.28551059717 7411965.370...,POINT (365292.5278944911 7411632.474641845),Manutenção,1
...,...,...,...,...,...,...,...
77994,B,355645305000181,35564530500,"POLYGON ((295845.4767163223 7387718.171744677,...",POINT (295635.6267752948 7387987.463104),Criação,1
77995,B,355645305000182,35564530500,POLYGON ((295659.08342691767 7387724.646166599...,POINT (295346.32280893414 7387476.172532191),Criação,1
77996,B,355645305000183,35564530500,"POLYGON ((295717.9367855333 7387400.28120076, ...",POINT (295595.4531606903 7387463.478657244),Criação,1
77997,B,355645305000184,35564530500,POLYGON ((295747.01881714363 7383804.520727156...,POINT (295440.169545104 7383369.551874219),Criação,1


In [9]:
df['id'].apply(lambda x: G_compat.edges([x]))#.pivot_table(index='id', columns='classe', values='nome', aggfunc='count').fillna(0).reset_index()

KeyError: 'id'

In [ ]:
pd.DataFrame([i[1] for i in G_compat.edges()])

""


In [14]:
G_compat.edge_gdf.pivot_table(index='metodo', values='geometry', aggfunc='count').to_dict()['geometry']

{'Divisão': 22600,
 'Manutenção': 18128,
 'Manutenção desassociada': 749,
 'Sobreposição (-10m)': 1845,
 'Sobreposição (-20m)': 6537,
 'Sobreposição (0m)': 422}

In [18]:
G_compat.node_gdf

,nome,malha,classe,group,grau,center
0,350390105000001,A,Exclusão,35039010500,2,POINT (365067.632 7412047.847)
1,350390105000002,A,Manutenção,35039010500,1,POINT (364973.206 7412437.846)
2,350390105000003,A,Manutenção,35039010500,1,POINT (365357.711 7412377.231)
3,350390105000004,A,Manutenção,35039010500,1,POINT (365482.285 7411732.847)
4,350390105000005,A,Manutenção,35039010500,1,POINT (365292.528 7411632.475)
...,...,...,...,...,...,...
77994,355645305000181,B,Criação,35564530500,1,POINT (295635.627 7387987.463)
77995,355645305000182,B,Criação,35564530500,1,POINT (295346.323 7387476.173)
77996,355645305000183,B,Criação,35564530500,1,POINT (295595.453 7387463.479)
77997,355645305000184,B,Criação,35564530500,1,POINT (295440.17 7383369.552)


In [ ]:
# Manutenções inconsistentes
len(G_compat.node_gdf.query('classe == "Manutenção" & grau > 1'))

120

In [ ]:
# Desconexos
len(G_compat.node_gdf.query('grau == 0'))

2

In [ ]:
# Razão de divisões não puras
df = (G_compat.edge_gdf.merge(G_compat.compatTable_B, left_on='B.nome', right_on='ID', how='left')
                       .merge(G_compat.AMC.reset_index()[['CD_PERIMETRO', 'TIPO_CORRESP']], on='CD_PERIMETRO', how='left' )
                       .pivot_table(index='TIPO_CORRESP', columns='metodo', values='B.nome', aggfunc='count'))

float(df.loc['m:n','Divisão']*100/df['Divisão'].sum())

18.787610619469028

In [ ]:
# Isolados persistentes
df = (G_compat.edge_gdf.merge(G_compat.compatTable_B, left_on='B.nome', right_on='ID', how='left')
                       .merge(G_compat.AMC.reset_index()[['CD_PERIMETRO', 'TIPO_CORRESP']], on='CD_PERIMETRO', how='left' ))

len(df.query('metodo == "Sobreposição (0m)"'))

422

In [ ]:
# Redesenhos extensos
(G_compat.edge_gdf.merge(G_compat.compatTable_B, left_on='B.nome', right_on='ID', how='left')
                  .merge(G_compat.AMC.reset_index()[['CD_PERIMETRO', 'TIPO_CORRESP']], on='CD_PERIMETRO', how='left' ))

,A.nome,A.classe,B.nome,B.classe,metodo,geometry,CD_PERIMETRO,ID,TIPO_CORRESP
0,350390105000001,Exclusão,350390105000169,Criação,Divisão,"LINESTRING (365067.632 7412047.847, 364934.18 ...",3503901050000001,350390105000169,1:n
1,350390105000001,Exclusão,350390105000170,Criação,Divisão,"LINESTRING (365067.632 7412047.847, 365314.203...",3503901050000001,350390105000170,1:n
2,350390105000002,Manutenção,350390105000002,Manutenção,Manutenção,"LINESTRING (364973.206 7412437.846, 364973.206...",3503901050000002,350390105000002,1:1
3,350390105000003,Manutenção,350390105000003,Manutenção,Manutenção,"LINESTRING (365357.711 7412377.231, 365357.712...",3503901050000003,350390105000003,1:1
4,350390105000004,Manutenção,350390105000004,Manutenção,Manutenção,"LINESTRING (365482.285 7411732.847, 365482.285...",3503901050000004,350390105000004,1:1
...,...,...,...,...,...,...,...,...,...
50276,355645305000060,Exclusão,355645305000178,Criação,Divisão,"LINESTRING (296246.239 7382888.239, 296250.104...",3556453050000052,355645305000178,1:n
50277,355645305000061,Exclusão,355645305000151,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 296724.54 7...",3556453050000053,355645305000151,1:n
50278,355645305000061,Exclusão,355645305000152,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 296632.168 ...",3556453050000053,355645305000152,1:n
50279,355645305000061,Exclusão,355645305000153,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 297157.34 7...",3556453050000053,355645305000153,1:n


In [ ]:
G_compat.edge_gdf

,A.nome,A.classe,B.nome,B.classe,metodo,geometry
0,350390105000001,Exclusão,350390105000169,Criação,Divisão,"LINESTRING (365067.632 7412047.847, 364934.18 ..."
1,350390105000001,Exclusão,350390105000170,Criação,Divisão,"LINESTRING (365067.632 7412047.847, 365314.203..."
2,350390105000002,Manutenção,350390105000002,Manutenção,Manutenção,"LINESTRING (364973.206 7412437.846, 364973.206..."
3,350390105000003,Manutenção,350390105000003,Manutenção,Manutenção,"LINESTRING (365357.711 7412377.231, 365357.712..."
4,350390105000004,Manutenção,350390105000004,Manutenção,Manutenção,"LINESTRING (365482.285 7411732.847, 365482.285..."
...,...,...,...,...,...,...
50276,355645305000060,Exclusão,355645305000178,Criação,Divisão,"LINESTRING (296246.239 7382888.239, 296250.104..."
50277,355645305000061,Exclusão,355645305000151,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 296724.54 7..."
50278,355645305000061,Exclusão,355645305000152,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 296632.168 ..."
50279,355645305000061,Exclusão,355645305000153,Criação,Divisão,"LINESTRING (296833.77 7382886.121, 297157.34 7..."
